In [ ]:
# File to create full dataset with all variables based on selected entries

In [ ]:
#Load packages
import pyreadr
import pandas as pd
import numpy as np
from IPython.display import display 

In [ ]:
# Load created function
from utils import add_latest_entry_before_date, add_latest_diagnosis_entry

In [ ]:
# Read-in dataframe generated in R: "Approach including Trend of HbA1c"
# input_data_trend has been uploaded --> 227k entries 
input = pyreadr.read_r("/home/jupyter-niclas/input_data_trend_NEW.rds")
input_data_trend = next(iter(input.values()))
# Renaming of column
input_data_trend = input_data_trend.rename(columns={'val': 'val_HbA1c'})
display(input_data_trend)

In [ ]:
# Adding column sexe to input_data_trend according to data_poblacio
poblacio = pyreadr.read_r("/home/dadesSP_berta75/taulesSIDIAP/taulesSIDIAP/ERC_ERC_DM_entregable_poblacio_20240409_202620.rds")
data_poblacio = next(iter(poblacio.values()))
input_data_trend = input_data_trend.merge(data_poblacio[['idp', 'sexe']], on='idp', how='left')
# Check new column for number of NAs
sexe_NA = input_data_trend['sexe'].isna().sum()
print (sexe_NA)

In [ ]:
# Adding column birth_date to input_data_trend according to dnaix from data_poblacio
input_data_trend = input_data_trend.merge(data_poblacio[['idp', 'dnaix']], on='idp', how='left')
# Check new column for number of NAs
dnaix_NA = input_data_trend['dnaix'].isna().sum()
print (dnaix_NA)

In [ ]:
# Using onset-date and birth date to determine age at t0
# Convert to datetime format
input_data_trend['dat'] = pd.to_datetime(input_data_trend['dat'].astype(str), format='%Y%m%d')
input_data_trend['dnaix'] = pd.to_datetime(input_data_trend['dnaix'].astype(str), format='%Y%m%d')
# Calculate age (in years)
input_data_trend['age'] = (input_data_trend['dat'] - input_data_trend['dnaix']).dt.days // 365
# Exclude individuals younger than 18
input_data_trend = input_data_trend[input_data_trend['age'] >= 18]
# Check for NAs in age
age_NA = input_data_trend['age'].isna().sum()
print(f"Number of missing age entries: {age_NA}")
display (input_data_trend)

In [ ]:
# Adding column smoking status from tabaquisme
tabaquisme = pyreadr.read_r("/home/dadesSP_berta75/taulesSIDIAP/taulesSIDIAP/ERC_ERC_DM_entregable_tabaquisme_20240409_202620.rds")
data_tabaquisme = next(iter(tabaquisme.values()))
# Converting code into smoking status
smoking_mapping = {
    0: "No smoker",
    1: "Smoker",
    2: "Ex-smoker"
}
data_tabaquisme['val'] = data_tabaquisme['val'].replace(smoking_mapping)
data_tabaquisme.rename(columns={'val': 'smoking_status'}, inplace=True)

In [ ]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_tabaquisme,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='smoking_status'   
)

In [ ]:
# Adding column alcohol risk consumption from variables_cliniques --> subset ALRIS generated in R
alris = pyreadr.read_r("Data_preparation/data_ALRIS.rds")
data_alris = next(iter(alris.values()))

In [ ]:
# Converting code into smoking status
alc_ris_mapping = {
    0: "No risk consumption",
    1: "Low risk consumption",
    2: "High risk consumption"
}
data_alris['val'] = data_alris['val'].replace(alc_ris_mapping)

In [ ]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_alris,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='alcohol_risk_consumption'
)

In [ ]:
# Adding MEDEA index from variables_socioeconomiques
var_socioeco = pyreadr.read_r("/home/dadesSP_berta75/taulesSIDIAP/taulesSIDIAP/ERC_ERC_DM_entregable_variables_socioeconomiques_20240409_202620.rds")
data_var_socioeco = next(iter(var_socioeco.values()))
# Add both columns to dataframe
input_data_trend = input_data_trend.merge(data_var_socioeco[['idp', 'ruralitat', 'qmedea']], on='idp', how='left')
# Check new column for number of NAs
# Convert empty strings or spaces to NaN
input_data_trend['ruralitat'] = input_data_trend['ruralitat'].replace(r'^\s*$', np.nan, regex=True)
ruralitat_NA = input_data_trend['ruralitat'].isna().sum()
print (ruralitat_NA)
# Check new column for number of NAs
# Convert empty strings or spaces to NaN
input_data_trend['qmedea'] = input_data_trend['qmedea'].replace(r'^\s*$', np.nan, regex=True)
qmedea_NA = input_data_trend['qmedea'].isna().sum()
print (qmedea_NA)
display(input_data_trend)

In [ ]:
# Start Block Comorbidities
# creating dataframe data_diagnostics
diagnostics = pyreadr.read_r("Data_preparation/data_both_diagnostics.rds")
data_diagnostics = next(iter(diagnostics.values()))

In [ ]:
display(data_diagnostics)

In [ ]:
#hypertension
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['I10', 'I11', 'I12', 'I13', 'I14', 'I15'],
    reference_date_col='dat',
    result_col_name='hypertension'
)

In [ ]:
# dyslipidemia
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['E78'],
    reference_date_col='dat',
    result_col_name='dyslipidemia'
)

In [ ]:
# Heart failure
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['I50'],
    reference_date_col='dat',
    result_col_name='heart_failure'
)

In [ ]:
# Peripheral artery disease
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    fixed_codes=['I73.9'],
    reference_date_col='dat',
    result_col_name='peripheral_artery_disease'
)

In [ ]:
# stroke
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['I61', 'I62', 'I63'],
    reference_date_col='dat',
    result_col_name='stroke'
)

In [ ]:
# ischemic heart disease
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['I20', 'I21', 'I22', 'I23','I24', 'I25'],
    reference_date_col='dat',
    result_col_name='ischemic_heart_disease'
)

In [ ]:
# Chronic kidney disease
input_data_trend = add_latest_diagnosis_entry(
    goal_df=input_data_trend,
    diagnostics_df=data_diagnostics,
    code_col='cod',
    date_col='dat',
    code_roots= ['N18', 'I12', 'I13'],
    reference_date_col='dat',
    result_col_name='chronic_kidney_disease'
)

In [ ]:
#END section comorbidities
display(input_data_trend)

In [ ]:
# Adding column BMI from variables cliniques --> subset BMI generated in R
BMI = pyreadr.read_r("Data_preparation/data_BMI.rds")
data_BMI = next(iter(BMI.values()))

In [ ]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_BMI,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='BMI'
)

In [ ]:
# Adding column Systolic blood pressure from variables cliniques --> subset SysBP generated in R
SysBP = pyreadr.read_r("Data_preparation/data_SysBP.rds")
data_SysBP = next(iter(SysBP.values()))

In [ ]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_SysBP,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='systolic_blood_pressure'
)

In [ ]:
# Adding column Diastolic blood pressure from variables cliniques --> subset DiasBP generated in R
DiasBP = pyreadr.read_r("Data_preparation/data_DiasBP.rds")
data_DiasBP = next(iter(DiasBP.values()))

In [ ]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_DiasBP,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='diastolic_blood_pressure'
)

In [ ]:
# Adding column cholesterol LDL from variables analitiques --> subset cLDL generated in R
cLDL = pyreadr.read_r("Data_preparation/data_cLDL.rds")
data_cLDL = next(iter(cLDL.values()))

In [ ]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_cLDL,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='cholesterol_LDL'
)

In [ ]:
# Adding column triglycerides from variables analitiques --> subset trigly generated in R
trigly = pyreadr.read_r("Data_preparation/data_trigly.rds")
data_trigly = next(iter(trigly.values()))

In [ ]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_trigly,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='triglycerides'
)

In [ ]:
# Adding column eGFR from variables analitiques --> subset eGFR generated in R
eGFR = pyreadr.read_r("Data_preparation/data_eGFR.rds")
data_eGFR = next(iter(eGFR.values()))

In [ ]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_eGFR,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='eGFR'
)

In [ ]:
# Adding column ACR from variables cliniques --> subset ACR generated in R
ACR = pyreadr.read_r("Data_preparation/data_ACR.rds")
data_ACR = next(iter(ACR.values()))

In [ ]:
input_data_trend = add_latest_entry_before_date(
    goal_df=input_data_trend,
    data_df=data_ACR,
    source_date_col='dat',          
    reference_date_col='dat',        
    value_col='val' ,
    result_col_name='ACR'
)

In [ ]:
display(input_data_trend)

In [ ]:
# Save generated input_data_trend-file
input_data_trend.to_csv("generated_input_data_trend_NEW.csv", index=False)